In [9]:
import os
import boto3
import duckdb
from dotenv import load_dotenv
from scripts.utils import download_object, get_all_objects

# Para probar que podamos conectarnos al bucket correctamente

In [10]:
load_dotenv()  # take environment variables from .env.

True

In [11]:
s3 = boto3.client('s3',
    endpoint_url='https://'+os.getenv('DUCKDB_S3_ENDPOINT'),
)

In [12]:
print('Probando desde duckdb:')
res = duckdb.sql(
'''
select *
from read_csv_auto('s3://archiva-apagones/penguins.csv')   
''')
print(res)

Probando desde duckdb:
┌─────────┬───────────┬────────────────┬───────────────┬───────────────────┬─────────────┬─────────┐
│ species │  island   │ bill_length_mm │ bill_depth_mm │ flipper_length_mm │ body_mass_g │   sex   │
│ varchar │  varchar  │     double     │    double     │       int64       │    int64    │ varchar │
├─────────┼───────────┼────────────────┼───────────────┼───────────────────┼─────────────┼─────────┤
│ Adelie  │ Torgersen │           39.1 │          18.7 │               181 │        3750 │ MALE    │
│ Adelie  │ Torgersen │           39.5 │          17.4 │               186 │        3800 │ FEMALE  │
│ Adelie  │ Torgersen │           40.3 │          18.0 │               195 │        3250 │ FEMALE  │
│ Adelie  │ Torgersen │           NULL │          NULL │              NULL │        NULL │ NULL    │
│ Adelie  │ Torgersen │           36.7 │          19.3 │               193 │        3450 │ FEMALE  │
│ Adelie  │ Torgersen │           39.3 │          20.6 │            

In [13]:
# Parameters for function
local_bucket_path = './archiva-apagones' # Probs more useful as env var
prefix = 'regions_without_service'

# Create local bucket path, if not already there
os.makedirs(local_bucket_path, exist_ok=True)

# Get list of objects in bucket
obj_list = list(get_all_objects(s3, prefix))
bucket_obj_keys = [obj['Key'] for obj in obj_list]

# Get list of objects locally
local_obj_keys = []
for root, dirs, files in os.walk(local_bucket_path):
    for file in files:
        # Add to local_obj_list but strip off the local_bucket_path
        obj_key = os.path.join(root, file).replace(local_bucket_path, '')[1:]
        local_obj_keys.append(obj_key)

# Determine which (prefixed) objects are missing locally
missing_keys = set(bucket_obj_keys).difference(set(local_obj_keys))
print(f'{len(missing_keys)} objects missing locally')

# Download missing objects
for obj_key in missing_keys:
    local_path = os.path.join(local_bucket_path, obj_key)
    download_object(s3, obj_key, local_path)

2 objects missing locally


In [18]:
duckdb.execute(
'''
create or replace table raw_regions_without_service as (
    select *
    -- from read_json('s3://archiva-apagones/regions_without_service/2023-12-19/*.json', filename=true, auto_detect=true, format='auto') 
    from read_json('./archiva-apagones/regions_without_service/*/*.json', filename=true, auto_detect=true, format='auto') 
)
''')
# select * from read_json('samples/regions_without_service/*.json', filename=true, auto_detect=true, format='auto')

In [19]:
duckdb.sql(
'''
describe
    select
        *
    from raw_regions_without_service
''')

┌─────────────┬────────────────────────────────────────────────────────────────┬─────────┬─────────┬─────────┬─────────┐
│ column_name │                          column_type                           │  null   │   key   │ default │  extra  │
│   varchar   │                            varchar                             │ varchar │ varchar │ varchar │ varchar │
├─────────────┼────────────────────────────────────────────────────────────────┼─────────┼─────────┼─────────┼─────────┤
│ regions     │ STRUCT("name" VARCHAR, percentageClientsWithService DOUBLE, …  │ YES     │ NULL    │ NULL    │ NULL    │
│ timestamp   │ VARCHAR                                                        │ YES     │ NULL    │ NULL    │ NULL    │
│ totals      │ STRUCT(totalClients BIGINT, totalClientsWithService BIGINT, …  │ YES     │ NULL    │ NULL    │ NULL    │
│ filename    │ VARCHAR                                                        │ YES     │ NULL    │ NULL    │ NULL    │
└─────────────┴─────────────────

In [20]:
duckdb.sql(
'''
    select
        strptime("timestamp", '%m/%d/%Y %I:%M %p') as "timestamp",
        totals.totalClients as clientes,
        totals.totalClientsWithService as clientes_con_servicio,
        totals.totalClientsWithoutService as clientes_sin_servicio,
        filename
            .string_split('__')[2]
            .regexp_extract('(.*).json', 1)
            as uhh
    from raw_regions_without_service
''')

┌─────────────────────┬──────────┬───────────────────────┬───────────────────────┬──────────────────────────┐
│      timestamp      │ clientes │ clientes_con_servicio │ clientes_sin_servicio │           uhh            │
│      timestamp      │  int64   │         int64         │         int64         │         varchar          │
├─────────────────────┼──────────┼───────────────────────┼───────────────────────┼──────────────────────────┤
│ 2023-12-25 11:50:00 │  1468223 │               1467888 │                   335 │ 2023-12-25T11-50-18-0400 │
│ 2023-12-25 05:25:00 │  1468223 │               1467376 │                   847 │ 2023-12-25T05-25-18-0400 │
│ 2023-12-25 13:30:00 │  1468223 │               1467814 │                   409 │ 2023-12-25T13-30-20-0400 │
│ 2023-12-25 21:30:00 │  1468223 │               1468160 │                    63 │ 2023-12-25T21-30-18-0400 │
│ 2023-12-25 02:55:00 │  1468223 │               1467412 │                   811 │ 2023-12-25T02-55-18-0400 │
│ 2023-12-

In [7]:
duckdb.sql(
'''
describe
    select
        regions[1]
    from raw_regions_without_service
''')

┌─────────────┬────────────────────────────────────────────────────────────────┬─────────┬─────────┬─────────┬─────────┐
│ column_name │                          column_type                           │  null   │   key   │ default │  extra  │
│   varchar   │                            varchar                             │ varchar │ varchar │ varchar │ varchar │
├─────────────┼────────────────────────────────────────────────────────────────┼─────────┼─────────┼─────────┼─────────┤
│ regions[1]  │ STRUCT("name" VARCHAR, percentageClientsWithService DOUBLE, …  │ YES     │ NULL    │ NULL    │ NULL    │
└─────────────┴────────────────────────────────────────────────────────────────┴─────────┴─────────┴─────────┴─────────┘

In [8]:
duckdb.sql(
'''
    select
        regions[1],
        "timestamp",
        totals,
        filename
    from raw_regions_without_service
''')

┌──────────────────────┬─────────────────────┬──────────────────────┬──────────────────────────────────────────────────┐
│      regions[1]      │      timestamp      │        totals        │                     filename                     │
│ struct("name" varc…  │       varchar       │ struct(totalclient…  │                     varchar                      │
├──────────────────────┼─────────────────────┼──────────────────────┼──────────────────────────────────────────────────┤
│ {'name': Arecibo, …  │ 12/19/2023 12:40 AM │ {'totalClients': 1…  │ s3://archiva-apagones/regions_without_service/…  │
│ {'name': Arecibo, …  │ 12/19/2023 12:50 AM │ {'totalClients': 1…  │ s3://archiva-apagones/regions_without_service/…  │
│ {'name': Arecibo, …  │ 12/19/2023 09:55 AM │ {'totalClients': 1…  │ s3://archiva-apagones/regions_without_service/…  │
│ {'name': Arecibo, …  │ 12/19/2023 10:05 AM │ {'totalClients': 1…  │ s3://archiva-apagones/regions_without_service/…  │
│ {'name': Arecibo, …  │ 12/19/2